In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glider import Glider
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
import pickle
from stable_baselines3.common.env_checker import check_env
from matplotlib.patches import Ellipse

/Users/iandavenport/Desktop/research/projects/gliding/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
glider = Glider(u0=0.0, v0=0.01, w0=0.0, rho_s=1, rho_f=-10)

checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path="./big_state_models/",
    name_prefix="rl_model",
)

/Users/iandavenport/Desktop/research/projects/gliding/venv/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [3]:
for i in range(50):
    glider.forward(0)

In [4]:
glider.y

[0,
 3.6697145605062456e-05,
 -0.0042582120377297784,
 -0.009191012258352775,
 -0.018955688309654564,
 -0.031482392180825845,
 -0.043849555025821996,
 -0.06550005835101169,
 -0.07854121703247863,
 -0.11030904183310544,
 -0.1225009564573554,
 -0.16502487554490164,
 -0.1750983340715934,
 -0.22872969462068432,
 -0.23563495376163004,
 -0.30047970752896525,
 -0.30337640759140927,
 -0.37758091557127194,
 -0.45752307846000684,
 -0.5425120563691556,
 -0.6319042200133244,
 -0.72511082991023,
 -0.8216015783938322,
 -0.9209049224205682,
 -1.0226060978137677,
 -1.1263435924009952,
 -1.2318047097560405,
 -1.3387207065309823,
 -1.446861851896821,
 -1.5560326453391629,
 -1.66606734077922,
 -1.7768258589977546,
 -1.888190123276554,
 -2.0000608212279296,
 -2.1123545753511395,
 -2.2250014928399198,
 -2.3379430590735737,
 -2.451130337137198,
 -2.5645224362132337,
 -2.6780852137468996,
 -2.7917901792031246,
 -2.9056135705249697,
 -3.0195355777641577,
 -3.1335396915928233,
 -3.2476121574135655,
 -3.3617415

In [ ]:
model = PPO("MlpPolicy", glider, verbose=1, tensorboard_log="big_state_logs/")
model.learn(total_timesteps=5e5, callback=checkpoint_callback)

In [ ]:
glider = Glider(u0=0.25, v0=-0.25, w0=0.1)
model = PPO.load("big_state_models/rl_model_500000_steps.zip", env=glider)
done = False
obs = glider.reset()
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, _ = glider.step(action.item())

In [ ]:
plt.plot(glider.theta)

In [ ]:
plt.scatter(glider.x, glider.y)

In [ ]:
plt.plot(glider.beta)

In [ ]:
def save_history(glider: Glider, filename: str) -> None:
    """
    Save the full history of the glider in a dictionary
    """
    history = {
        "u": glider.u,
        "v": glider.v,
        "w": glider.w,
        "x": glider.x,
        "y": glider.y,
        "theta": glider.theta,
        "beta": glider.beta,
    }
    with open(filename + ".pkl", "wb") as f:
        pickle.dump(history, f)

In [ ]:
save_history(glider=glider, filename="history")

In [ ]:
# ell = Ellipse(
#         xy = (1, 1),
#         width = 1,
#         height = 0.3,
#         angle = np.rad2deg(np.pi/2.5)
#         )
# fig, ax = plt.subplots()
# ax.add_artist(ell)
# ax.set_xlim(-5, 5)
# ax.set_ylim(-5, 5)
# plt.show()